<a href="https://colab.research.google.com/github/PatelVatsalB21/AI-Road-Semantic-Segmentation/blob/test/AI_Semantic_Segmantation_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
import random
from tqdm import notebook
from multiprocessing import Pool

from tensorflow import keras
from keras.models import Sequential, Model
from keras.layers import *
from keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
import keras.backend as K
import tensorflow as tf
from keras.utils.vis_utils import plot_model
from keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.regularizers import l2

np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
def plotter(img,mask):
    fig,axes=plt.subplots(1,2)
    axes[0].imshow(img)
    plt.imshow(mask)
  

In [ ]:
input_images = []
mask_images = []

image_path = "../content/CameraRGB/"
mask_path = "../content/CameraSeg/"
image_list = os.listdir(image_path)
mask_list = os.listdir(mask_path)
image_list = [image_path + i for i in image_list]
mask_list = [mask_path + i for i in mask_list]

for i in notebook.tqdm(range(len(image_list))):
    img, mask = cv2.imread(image_list[i]), cv2.imread(mask_list[i])[:, :, 2]
    img = cv2.resize(img, (256, 256))
    mask = cv2.resize(mask, (256, 256,))
    input_images.append(img)
    mask_images.append(mask)

input_images = np.array(input_images)
input_images = input_images / 255.
mask_images = np.array(mask_images)
mask_images = mask_images.reshape((mask_images.shape[0], mask_images.shape[1], mask_images.shape[2], 1))
input_img = input_images[0]
# h = input_img.shape[0]
# w = input_img.shape[1]
# l = input_img.shape[2]

In [ ]:
print("started pipeline")
input_images = []
mask_images = []

def process_data():
    print("process_data")
    image_path = "../content/CameraRGB/"
    mask_path = "../content/CameraSeg/"
    image_list = os.listdir(image_path)
    mask_list = os.listdir(mask_path)
    image_list = [image_path + i for i in image_list]
    mask_list = [mask_path + i for i in mask_list]


    for i in notebook.tqdm(range(1000)):
        img,mask= cv2.imread(image_list[i]),cv2.imread(mask_list[i])[:,:,2]
        img=cv2.resize(img,(256,256))
        mask=cv2.resize(mask,(256,256,))
        input_images.append(img)
        mask_images.append(mask)

process_data()

input_images = np.array(input_images)
input_images = input_images/255.
mask_images = np.array(mask_images)
mask_images = mask_images.reshape((mask_images.shape[0], mask_images.shape[1], mask_images.shape[2], 1))

print(mask_images[0])
plt.imshow(mask_images[0])
plt.show()

print(input_images.shape, mask_images.shape)


input_img = input_images[0]
output_img = mask_images[0]

img_ht = input_img.shape[0]
img_wd = input_img.shape[1]
img_ly = input_img.shape[2]

print(img_ht, img_wd, img_ly)

In [ ]:
    inputs = Input(shape=(img_ht,img_wd,img_ly), name = 'img_input')
    x = Conv2D(32, 3, strides=2, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    previous_block_activation = x
    for filters in [64, 128, 256]:
        x = Activation("relu")(x)
        x = SeparableConv2D(filters, 3, padding="same")(x)
        x = BatchNormalization()(x)

        x = Activation("relu")(x)
        x = SeparableConv2D(filters, 3, padding="same")(x)
        x = BatchNormalization()(x)

        x = MaxPooling2D(3, strides=2, padding="same")(x)

     
        residual = Conv2D(filters, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = keras.layers.add([x, residual]) 
        previous_block_activation = x 
    for filters in [256, 128, 64, 32]:
        x = Activation("relu")(x)
        x = Conv2DTranspose(filters, 3, padding="same")(x)
        x = BatchNormalization()(x)

        x = Activation("relu")(x)
        x = Conv2DTranspose(filters, 3, padding="same")(x)
        x = BatchNormalization()(x)

        x = UpSampling2D(2)(x)

        residual = UpSampling2D(2)(previous_block_activation)
        residual = Conv2D(filters, 1, padding="same")(residual)
        x = keras.layers.add([x, residual])  
        previous_block_activation = x 

    outputs = Conv2D(num_classes, 3, activation="sigmoid", padding="same")(x)

    model = Model(inputs = [inputs], outputs = [outputs])

In [ ]:
keras.backend.clear_session()
model = get_model((256,256), 13)
model.compile(optimizer="rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
callbacks = [
        ReduceLROnPlateau(monitor="val_loss", patience=3, factor=0.1, verbose=1, min_lr=1e-12),
    ]

In [ ]:
history = model.fit(input_images, mask_images, batch_size=32, epochs = 10, callbacks = callbacks, validation_split = 0.1)

In [ ]:
history.history

In [ ]:
plt.figure()
plt.plot(history.history['accuracy'], label = 'Training')
plt.plot(history.history['val_accuracy'], label = 'Validation')
plt.title('Accuracy')
plt.legend()

plt.figure()
plt.plot(history.history['loss'], label = 'Training')
plt.plot(history.history['val_loss'], label = 'Validation')
plt.title('loss')
plt.legend()

In [ ]:
model.save("/content/semantic_model.h5")

In [ ]:
test = input_images[0]
getshape=test.shape
pred = model.predict(test.reshape(1,getshape[0],getshape[1],getshape[2]))

print(pred.shape)        
newimg = np.zeros((256,256))
print(pred)
for i in range(13):
    for j in range(256):
        for k in range(256):
            if pred[0,j,k,i] > 0.5:
                newimg[j,k] = i

plotter(test,newimg)
# plt.imshow(newimg, cmap="Paired")
plotter(test,input_images[0])